# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.92it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.65it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.30it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.18it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.32it/s]



...

 78%|███████▊  | 18/23 [00:04<00:01,  4.51it/s]

 83%|████████▎ | 19/23 [00:05<00:00,  4.52it/s]

 87%|████████▋ | 20/23 [00:05<00:00,  4.60it/s]

 91%|█████████▏| 21/23 [00:05<00:00,  4.52it/s]

100%|██████████| 23/23 [00:05<00:00,  3.92it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Ben, and I'm a proud husband and father of two young girls. My wife, Rachel, and I met in college and have been married for over 10 years. We have two amazing daughters, Emily and Abigail, who are the lights of our lives. We live in the beautiful state of Colorado and love the outdoors. When I'm not working, you can find me hiking, camping, or skiing in the mountains. I'm passionate about helping others and making a positive impact on my community. I'm excited to share my story and connect with like-minded individuals.
Hello, my name is Ben, and I'm a proud
Prompt: The president of the United States is
Generated text:  not bound by the same rules as the rest of us. While we have to follow the law, the president can often operate with a degree of impunity, thanks to the Constitution's "executive privilege" doctrine.
Here's a rundown of what executive privilege is and how it's used:
What is executive privilege?
Executive privilege is a legal doc

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Am

ara

 Kar

an

...

:


1

.

 SO

AR

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Mark. I am a passionate techie, gamer, and learner. I have a strong background in computer programming and software development. I enjoy solving complex problems, learning new things, and staying up to date with the latest technologies.

Here are some of the things I'm interested in:
- **Programming languages**: I'm proficient in C++, Java, Python, and JavaScript. I enjoy working with different programming languages and exploring their unique features and applications.
- **Game development**: I have a strong passion for game development and enjoy creating games using various game engines such as Unity and Unreal Engine.
- **Artificial intelligence and machine learning**: I

Prompt: The capital of France is
Generated text:  a city of romance, art, fashion and history. As you walk through the streets of Paris, you'll see stunning architecture, world-class museums, and picturesque cafes. From the iconic Eiffel Tower to the charming Seine River, 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Joe

 P

esh

lak

...

 this

 has

 improved

 productivity

 and

In [6]:
llm.shutdown()